In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from numpy.random import randn
import pandas as pd
import os
from datetime import timedelta, date, datetime
from string import Template
#from pandas import ExcelWriter
#from pandas import ExcelFile
os.getcwd()

'E:\\winpython\\WinPython-32bit-3.4.3.4\\notebooks'

In [2]:
#更换目录
os.chdir('F:\\2015年\\客户工程\\光改\\bigdatecoming_光改周报\\监理上报\\output')
cwd = os.getcwd()
cwd

'F:\\2015年\\客户工程\\光改\\bigdatecoming_光改周报\\监理上报\\output'

In [3]:
#读取光改项目监理汇总明细表
df = pd.read_excel('监理汇总916.xlsx',sheetname='项目进度明细表（每日更新）')
#df.head()

In [32]:
#计划在昨天前完工的项目个数
df1 = df[df['计划完工时间'] <= datetime(2015,9,16)].groupby('分公司').count()[['计划完工时间']]
df1

,计划完工时间
分公司,
东莞,18
中山,13
佛山,20
广州,42
惠州,6
揭阳,1
汕尾,1
江门,2
深圳,17


In [5]:
#已完工项目个数
df2 = df.groupby('分公司').count()[['实际完工时间']]
df2

,实际完工时间
分公司,
东莞,20
中山,14
佛山,21
广州,30
惠州,10
揭阳,1
汕尾,1
江门,2
深圳,14


In [6]:
#合并计划完工数与实际完工数
dfmerge = pd.merge(df1,df2,how='left',left_index=True, right_index=True)
dfmerge = dfmerge.fillna(value=0)
dfmerge

,计划完工时间,实际完工时间
分公司,,
东莞,18,20
中山,13,14
佛山,20,21
广州,42,30
惠州,6,10
揭阳,1,1
汕尾,1,1
江门,2,2
深圳,17,14


In [33]:
#选出实际完工数小于计划完工数的地市
dfmergefault = dfmerge[dfmerge['实际完工时间'] < dfmerge['计划完工时间']]
dfmergefault 

,计划完工时间,实际完工时间
分公司,,
广州,42,30
深圳,17,14
肇庆,5,4
茂名,3,1


In [12]:
#添加一列落后计划个数 
dfmergename = ['计划完工时间','实际完工时间','落后计划数']
dfmergefault = dfmergefault.reindex(columns=dfmergename)
dfmergefault['落后计划数'] = (dfmergefault['计划完工时间'] - dfmergefault['实际完工时间'])
sumdata = dfmergefault.sum() #对每列求和
row_sum = pd.DataFrame(data = sumdata) #把系统转换为dataframe.
row_sum.T # 测试列转行
dfmergefault

,计划完工时间,实际完工时间,落后计划数
分公司,,,
广州,42,30,12
深圳,17,14,3
肇庆,5,4,1
茂名,3,1,2


In [13]:
# 列转行后存入原行变量
row_sum = row_sum.T
row_sum

,计划完工时间,实际完工时间,落后计划数
0,67,49,18


In [14]:
#把求和列添加到最后一列。
branchdata = pd.concat([dfmergefault,row_sum])
branchdata

,计划完工时间,实际完工时间,落后计划数
分公司,,,
广州,42,30,12
深圳,17,14,3
肇庆,5,4,1
茂名,3,1,2
0,67,49,18


In [15]:
#更改index。
branchdata = branchdata.rename(index ={0:'合计'})
branchdata

,计划完工时间,实际完工时间,落后计划数
分公司,,,
广州,42,30,12
深圳,17,14,3
肇庆,5,4,1
茂名,3,1,2
合计,67,49,18


In [16]:
#按需要把地市对应落后计划数放进字典。
luohouNumber = branchdata.ix['合计','落后计划数']
branchDict = {}
for index in branchdata.index:
    if index != '合计':
        branchDict[index] = branchdata.ix[index,'落后计划数']
branchDict

{'广州': 12, '深圳': 3, '肇庆': 1, '茂名': 2}

In [17]:
#实际完工数小于计划完工数的地市
cityfault = dfmerge[dfmerge['实际完工时间'] < dfmerge['计划完工时间']].index
city = []
for item in cityfault:
    city.append(item)
    
city

['广州', '深圳', '肇庆', '茂名']

In [18]:
#计划在昨天完工的项目个数-按监理
df3 = df[df['计划完工时间'] <= datetime(2015,9,15)].groupby('监理单位').count()[['计划完工时间']]
df3.sum()

计划完工时间    139
dtype: int64

In [19]:
df4 = df.groupby('监理单位').count()[['实际完工时间']]
df4.sum()

实际完工时间    137
dtype: int64

In [20]:
dfmergejl = pd.merge(df3,df4,how='left',left_index=True, right_index=True)
dfmergejl

,计划完工时间,实际完工时间
监理单位,,
中通服项目管理咨询有限公司,3,6
公诚管理咨询有限公司,34,44
华夏邮电咨询监理有限公司,24,21
广州市汇源通信建设监理有限公司,30,19
深圳市都信建设监理有限公司,16,13
达安监理,32,34


In [21]:
dfmergefaultjl = dfmergejl[dfmergejl['实际完工时间'] < dfmergejl['计划完工时间']]

In [22]:
dfmergenamejl = ['计划完工时间','实际完工时间','落后计划数']
dfmergefaultjl = dfmergefaultjl.reindex(columns=dfmergenamejl)
dfmergefaultjl['落后计划数'] = (dfmergefaultjl['计划完工时间'] - dfmergefaultjl['实际完工时间'])
sumdatajl = dfmergefaultjl.sum()
row_sumjl = pd.DataFrame(data = sumdatajl)
row_sumjl.T
dfmergefaultjl

,计划完工时间,实际完工时间,落后计划数
监理单位,,,
华夏邮电咨询监理有限公司,24,21,3
广州市汇源通信建设监理有限公司,30,19,11
深圳市都信建设监理有限公司,16,13,3


In [23]:
row_sumjl = row_sumjl.T
row_sumjl

,计划完工时间,实际完工时间,落后计划数
0,70,53,17


In [24]:
jldata = pd.concat([dfmergefaultjl,row_sumjl])
jldata

,计划完工时间,实际完工时间,落后计划数
监理单位,,,
华夏邮电咨询监理有限公司,24,21,3
广州市汇源通信建设监理有限公司,30,19,11
深圳市都信建设监理有限公司,16,13,3
0,70,53,17


In [25]:
jldata = jldata.rename(index = {0:'合计'})
jldata

,计划完工时间,实际完工时间,落后计划数
监理单位,,,
华夏邮电咨询监理有限公司,24,21,3
广州市汇源通信建设监理有限公司,30,19,11
深圳市都信建设监理有限公司,16,13,3
合计,70,53,17


In [26]:
jlDict = {}
for index in jldata.index:
    if index != '合计':
        jlDict[index] = jldata.ix[index,'落后计划数']
jlDict

{'华夏邮电咨询监理有限公司': 3, '广州市汇源通信建设监理有限公司': 11, '深圳市都信建设监理有限公司': 3}

In [27]:
df[df['计划完工时间'] == datetime(2015,9,16)][['计划完工时间','实际完工时间']]

,计划完工时间,实际完工时间
259,2015-09-16,2015-09-11


In [28]:
date.today() 

datetime.date(2015, 9, 16)

In [29]:
#新建一个列表，添加一个字典进列表，字典数据为需要打印数据。
dailydata = []
dailydata.append(dict(dateoftoday = date.today(),luohounumber=luohouNumber,branchdict=branchDict,jldict=jlDict))

In [30]:
#新建一个打印模板,然后打印
t_morningreport = Template("各地市分公司好：\n\n\t截止$dateoftoday，落后于计划的项目有$luohounumber个，其中$branchdict，分公司务必加快工程进度\
!以监理单位维度统计为：$jldict。\n\n\t\t\t\t\t广东“沃光改”推进办公室")
print(t_morningreport.substitute(dailydata[0]))

各地市分公司好：

	截止2015-09-16，落后于计划的项目有18个，其中{'广州': 12, '肇庆': 1, '茂名': 2, '深圳': 3}，分公司务必加快工程进度!以监理单位维度统计为：{'广州市汇源通信建设监理有限公司': 11, '华夏邮电咨询监理有限公司': 3, '深圳市都信建设监理有限公司': 3}。

					广东“沃光改”推进办公室
